<a href="https://colab.research.google.com/github/guanyu1127/Programming-Language/blob/main/HW3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [16]:
# =========================
# 0) 安裝套件
# =========================
!pip -q install gspread google-auth plotly gradio pandas python-dateutil pygithub

# =========================
# 1) 匯入 & Google 授權
# =========================
from google.colab import auth
auth.authenticate_user()

import gspread
from google.auth import default
import pandas as pd
from datetime import datetime
from dateutil import parser as dateparser
import json
import os
import plotly.express as px
import gradio as gr
import uuid

# =========================
# 2) Google 試算表：建立/開啟 + 工作表
# =========================
creds, _ = default()
gc = gspread.authorize(creds)

# 你可改成課程指定名稱，例如 "Python期末作業_"
SPREADSHEET_TITLE = "PomodoroTaskManager_Sheet"

def get_or_create_spreadsheet(title=SPREADSHEET_TITLE):
    try:
        sh = gc.open(title)
    except gspread.SpreadsheetNotFound:
        sh = gc.create(title)
    return sh

sh = get_or_create_spreadsheet()

TASKS_HEADER = ["id", "title", "tag", "due_date", "status", "created_at", "completed_at", "pomodoro_count", "total_focus_min"]
POMODOROS_HEADER = ["id", "task_id", "start_at", "end_at", "duration_min"]

def get_or_create_worksheet(sh, name, header):
    try:
        ws = sh.worksheet(name)
    except gspread.WorksheetNotFound:
        ws = sh.add_worksheet(title=name, rows=1, cols=len(header))
        ws.append_row(header)
    # 校驗表頭
    if ws.row_values(1) != header:
        ws.update('1:1', [header])
    return ws

ws_tasks = get_or_create_worksheet(sh, "tasks", TASKS_HEADER)
ws_pomos = get_or_create_worksheet(sh, "pomodoros", POMODOROS_HEADER)

def read_df(ws):
    data = ws.get_all_records()
    if not data:
        return pd.DataFrame(columns=TASKS_HEADER if ws.title=="tasks" else POMODOROS_HEADER)
    return pd.DataFrame(data)

def write_df(ws, df):
    df = df.copy()
    ws.clear()
    ws.update('1:1', [df.columns.tolist()])
    if len(df):
        ws.update('A2', df.astype(str).values.tolist())

tasks_df = read_df(ws_tasks)
pomos_df = read_df(ws_pomos)

# 以 list/dict 保存（作業需求）
tasks = tasks_df.to_dict(orient="records")
pomos = pomos_df.to_dict(orient="records")

def now_iso():
    return datetime.now().isoformat(timespec="seconds")

# =========================
# 3) 公用：安全日期解析
# =========================
def _safe_date_iso(s):
    """輸入 None/空字串/不合法都回傳空字串；合法才回 YYYY-MM-DD。"""
    if s is None:
        return ""
    s = str(s).strip()
    if not s:
        return ""
    try:
        return dateparser.parse(s).date().isoformat()
    except Exception:
        return ""  # 解析失敗就視為沒填

# =========================
# 4) 業務邏輯
# =========================
def sync_to_sheet():
    global tasks, pomos
    write_df(ws_tasks, pd.DataFrame(tasks, columns=TASKS_HEADER))
    write_df(ws_pomos, pd.DataFrame(pomos, columns=POMODOROS_HEADER))

def add_task(title, tag="", due_date=None):
    """新增任務（含安全檢查）"""
    global tasks
    title = "" if title is None else str(title)
    tag = "" if tag is None else str(tag)

    if not title.strip():
        raise ValueError("任務標題不可為空")

    tid = str(uuid.uuid4())[:8]
    rec = {
        "id": tid,
        "title": title.strip(),
        "tag": tag.strip(),
        "due_date": _safe_date_iso(due_date),
        "status": "todo",
        "created_at": now_iso(),
        "completed_at": "",
        "pomodoro_count": 0,
        "total_focus_min": 0
    }
    tasks.append(rec)
    sync_to_sheet()
    return tid

def delete_task(task_id):
    global tasks, pomos
    before = len(tasks)
    tasks = [t for t in tasks if t["id"] != task_id]
    pomos = [p for p in pomos if p["task_id"] != task_id]
    sync_to_sheet()
    return before - len(tasks)

def mark_done(task_id):
    for t in tasks:
        if t["id"] == task_id:
            t["status"] = "done"
            t["completed_at"] = now_iso()
            sync_to_sheet()
            return True
    return False

_active_sessions = {}  # {task_id: start_iso}

def start_pomodoro(task_id):
    if not task_id or not str(task_id).strip():
        return False, "請輸入任務ID。"
    if task_id in _active_sessions:
        return False, "該任務已在計時中。"
    _active_sessions[task_id] = now_iso()
    return True, f"任務 {task_id} 開始計時。"

def stop_pomodoro(task_id):
    global pomos, tasks
    if not task_id or task_id not in _active_sessions:
        return False, "該任務目前沒有在計時。"
    start_iso = _active_sessions.pop(task_id)
    start_dt = dateparser.parse(start_iso)
    end_dt = datetime.now()
    duration = max(1, int((end_dt - start_dt).total_seconds() // 60))
    pid = str(uuid.uuid4())[:8]
    pomos.append({
        "id": pid,
        "task_id": task_id,
        "start_at": start_dt.isoformat(timespec="seconds"),
        "end_at": end_dt.isoformat(timespec="seconds"),
        "duration_min": duration
    })
    for t in tasks:
        if t["id"] == task_id:
            t["pomodoro_count"] = int(t.get("pomodoro_count", 0)) + 1
            t["total_focus_min"] = int(t.get("total_focus_min", 0)) + duration
            break
    sync_to_sheet()
    return True, f"任務 {task_id} 已停止，紀錄 {duration} 分鐘。"

def query_tasks(status=None, start=None, end=None):
    def in_range(t):
        def ok(ts):
            if not ts: return False
            dt = dateparser.parse(ts)
            c1 = (start is None) or (dt >= start)
            c2 = (end is None) or (dt <= end)
            return c1 and c2
        if status == "done":
            return ok(t["completed_at"])
        if status is None:
            return ok(t["created_at"]) or ok(t["completed_at"])
        return ok(t["created_at"])

    result = tasks
    if status in ("todo", "done"):
        result = [t for t in result if t["status"] == status]
    if (start is not None) or (end is not None):
        result = [t for t in result if in_range(t)]
    return pd.DataFrame(result, columns=TASKS_HEADER)

# =========================
# 5) 匯出 / 匯入
# =========================
EXPORT_DIR = "/content/exports"
os.makedirs(EXPORT_DIR, exist_ok=True)

def export_csv_json(prefix="tasks_pomos"):
    tasks_df = pd.DataFrame(tasks, columns=TASKS_HEADER)
    pomos_df = pd.DataFrame(pomos, columns=POMODOROS_HEADER)

    tasks_csv = os.path.join(EXPORT_DIR, f"{prefix}_tasks.csv")
    pomos_csv = os.path.join(EXPORT_DIR, f"{prefix}_pomodoros.csv")
    tasks_json = os.path.join(EXPORT_DIR, f"{prefix}_tasks.json")
    pomos_json = os.path.join(EXPORT_DIR, f"{prefix}_pomodoros.json")

    tasks_df.to_csv(tasks_csv, index=False, encoding="utf-8-sig")
    pomos_df.to_csv(pomos_csv, index=False, encoding="utf-8-sig")
    with open(tasks_json, "w", encoding="utf-8") as f:
        json.dump(tasks, f, ensure_ascii=False, indent=2)
    with open(pomos_json, "w", encoding="utf-8") as f:
        json.dump(pomos, f, ensure_ascii=False, indent=2)
    return tasks_csv, pomos_csv, tasks_json, pomos_json

def import_csv(file_obj, target="tasks"):
    df = pd.read_csv(file_obj.name)
    recs = df.to_dict(orient="records")
    if target == "tasks":
        for r in recs:
            for k in TASKS_HEADER:
                r.setdefault(k, "" if k not in ("pomodoro_count","total_focus_min") else 0)
        globals()["tasks"][:] = recs
    else:
        for r in recs:
            for k in POMODOROS_HEADER:
                r.setdefault(k, "")
        globals()["pomos"][:] = recs
    sync_to_sheet()
    return f"已匯入 {len(recs)} 筆到 {target}。"

def import_json(file_obj, target="tasks"):
    recs = json.load(open(file_obj.name, "r", encoding="utf-8"))
    if target == "tasks":
        globals()["tasks"][:] = recs
    else:
        globals()["pomos"][:] = recs
    sync_to_sheet()
    return f"已匯入 {len(recs)} 筆到 {target}。"

# =========================
# 6) 視覺化統計（Plotly）
# =========================
def viz_summary():
    tasks_df = pd.DataFrame(tasks, columns=TASKS_HEADER)
    pomos_df = pd.DataFrame(pomos, columns=POMODOROS_HEADER)
    figs = []

    if len(tasks_df):
        s = tasks_df["status"].value_counts().reset_index()
        s.columns = ["status","count"]
        figs.append(px.pie(s, names="status", values="count", title="任務狀態分布"))
    else:
        figs.append(None)

    if len(pomos_df):
        pomos_df["day"] = pd.to_datetime(pomos_df["end_at"]).dt.date
        agg = pomos_df.groupby("day")["duration_min"].sum().reset_index()
        figs.append(px.line(agg, x="day", y="duration_min", title="每日番茄鐘總分鐘數"))
    else:
        figs.append(None)

    if len(tasks_df) and len(pomos_df):
        tag_map = {t["id"]: t["tag"] or "未分類" for t in tasks}
        pomos_df["tag"] = pomos_df["task_id"].map(tag_map)
        agg2 = pomos_df.groupby("tag")["duration_min"].sum().reset_index()
        figs.append(px.bar(agg2, x="tag", y="duration_min", title="各標籤累積專注時間(分)"))
    else:
        figs.append(None)

    return figs

# =========================
# 7) Gradio UI callbacks（含防呆）
# =========================
def ui_add_task(title, tag, due):
    try:
        if title is None or str(title).strip() == "":
            return "請輸入任務標題", pd.DataFrame(tasks, columns=TASKS_HEADER)
        tid = add_task(title, tag, due)
        return f"已新增任務：{tid}", pd.DataFrame(tasks, columns=TASKS_HEADER)
    except Exception as e:
        return f"新增失敗：{e}", pd.DataFrame(tasks, columns=TASKS_HEADER)

def ui_delete_task(task_id):
    if task_id is None or str(task_id).strip() == "":
        return "請輸入要刪除的任務ID", pd.DataFrame(tasks, columns=TASKS_HEADER)
    removed = delete_task(str(task_id).strip())
    msg = f"已刪除 {removed} 筆（含其番茄鐘紀錄）" if removed else "找不到該任務ID"
    return msg, pd.DataFrame(tasks, columns=TASKS_HEADER)

def ui_mark_done(task_id):
    if task_id is None or str(task_id).strip() == "":
        return "請輸入要標記完成的任務ID", pd.DataFrame(tasks, columns=TASKS_HEADER)
    ok = mark_done(str(task_id).strip())
    return ("標記完成成功" if ok else "找不到該任務ID"), pd.DataFrame(tasks, columns=TASKS_HEADER)

def ui_start_pomo(task_id):
    ok, msg = start_pomodoro(str(task_id).strip() if task_id else "")
    return msg

def ui_stop_pomo(task_id):
    ok, msg = stop_pomodoro(str(task_id).strip() if task_id else "")
    return msg, pd.DataFrame(tasks, columns=TASKS_HEADER)

def ui_query(status, start_date, end_date):
    sdt = dateparser.parse(start_date).replace(hour=0, minute=0, second=0) if start_date else None
    edt = dateparser.parse(end_date).replace(hour=23, minute=59, second=59) if end_date else None
    df = query_tasks(status if status!="all" else None, sdt, edt)
    return df

def ui_export(prefix):
    paths = export_csv_json(prefix or "tasks_pomos")
    return "已匯出：\n" + "\n".join(paths)

def ui_import_csv_tasks(file):
    msg = import_csv(file, "tasks")
    return msg, pd.DataFrame(tasks, columns=TASKS_HEADER)

def ui_import_csv_pomos(file):
    msg = import_csv(file, "pomos")
    return msg, pd.DataFrame(tasks, columns=TASKS_HEADER)

def ui_import_json_tasks(file):
    msg = import_json(file, "tasks")
    return msg, pd.DataFrame(tasks, columns=TASKS_HEADER)

def ui_import_json_pomos(file):
    msg = import_json(file, "pomos")
    return msg, pd.DataFrame(tasks, columns=TASKS_HEADER)

def ui_viz():
    figs = viz_summary()
    while len(figs) < 3:
        figs.append(None)
    return figs[0], figs[1], figs[2]

# =========================
# 8) Gradio 介面
# =========================
with gr.Blocks(title="番茄鐘任務管理器") as demo:
    gr.Markdown("## 番茄鐘任務管理器（Google Sheet + Gradio）")

    # ---- 任務管理 ----
    with gr.Tab("任務管理"):
        with gr.Row():
            title = gr.Textbox(label="任務標題", placeholder="例如：離散作業第3章")
            tag = gr.Textbox(label="標籤", placeholder="課業/專題/生活…")
            due = gr.Textbox(label="到期日(可空白)", placeholder="YYYY-MM-DD")
        add_btn = gr.Button("➕ 新增任務")
        add_msg = gr.Markdown()
        add_table = gr.Dataframe(label="任務表", value=pd.DataFrame(tasks, columns=TASKS_HEADER), interactive=False)

        with gr.Row():
            del_id = gr.Textbox(label="要刪除的任務ID")
            del_btn = gr.Button("🗑️ 刪除任務")
        del_msg = gr.Markdown()

        with gr.Row():
            did = gr.Textbox(label="要標記完成的任務ID")
            done_btn = gr.Button("✅ 標記完成")
        done_msg = gr.Markdown()

        add_btn.click(ui_add_task, [title, tag, due], [add_msg, add_table])
        del_btn.click(ui_delete_task, [del_id], [del_msg, add_table])
        done_btn.click(ui_mark_done, [did], [done_msg, add_table])

    # ---- 番茄鐘 ----
    with gr.Tab("番茄鐘"):
        with gr.Row():
            tid = gr.Textbox(label="任務ID（要計時的）")
        start_btn = gr.Button("▶️ 開始計時")
        stop_btn = gr.Button("⏹️ 停止計時並寫入")
        pomo_msg = gr.Markdown()
        table_after = gr.Dataframe(label="最新任務表", value=pd.DataFrame(tasks, columns=TASKS_HEADER), interactive=False)

        start_btn.click(ui_start_pomo, [tid], [pomo_msg])
        stop_btn.click(ui_stop_pomo, [tid], [pomo_msg, table_after])

    # ---- 查詢/篩選 ----
    with gr.Tab("查詢/篩選"):
        status = gr.Dropdown(["all","todo","done"], value="all", label="狀態")
        sdate = gr.Textbox(label="起始日期(可留空)", placeholder="YYYY-MM-DD")
        edate = gr.Textbox(label="結束日期(可留空)", placeholder="YYYY-MM-DD")
        q_btn = gr.Button("🔍 查詢")
        q_table = gr.Dataframe(label="查詢結果", interactive=False)
        q_btn.click(ui_query, [status, sdate, edate], [q_table])

    # ---- 匯出 / 匯入 ----
    with gr.Tab("匯出 / 匯入"):
        prefix = gr.Textbox(label="匯出檔名前綴", value="tasks_pomos")
        exp_btn = gr.Button("💾 匯出 CSV & JSON")
        exp_msg = gr.Markdown()

        gr.Markdown("### 匯入 CSV")
        csv_tasks = gr.File(label="tasks.csv")
        csv_pomos = gr.File(label="pomodoros.csv")
        imp_csv_tasks_btn = gr.Button("⬆️ 匯入 tasks.csv")
        imp_csv_pomos_btn = gr.Button("⬆️ 匯入 pomodoros.csv")
        imp_csv_msg = gr.Markdown()
        after_csv_table = gr.Dataframe(label="任務表（匯入後）", interactive=False)

        gr.Markdown("### 匯入 JSON")
        json_tasks = gr.File(label="tasks.json")
        json_pomos = gr.File(label="pomodoros.json")
        imp_json_tasks_btn = gr.Button("⬆️ 匯入 tasks.json")
        imp_json_pomos_btn = gr.Button("⬆️ 匯入 pomodoros.json")
        imp_json_msg = gr.Markdown()
        after_json_table = gr.Dataframe(label="任務表（匯入後）", interactive=False)

        exp_btn.click(ui_export, [prefix], [exp_msg])

        imp_csv_tasks_btn.click(ui_import_csv_tasks, [csv_tasks], [imp_csv_msg, after_csv_table])
        imp_csv_pomos_btn.click(ui_import_csv_pomos, [csv_pomos], [imp_csv_msg, after_csv_table])

        imp_json_tasks_btn.click(ui_import_json_tasks, [json_tasks], [imp_json_msg, after_json_table])
        imp_json_pomos_btn.click(ui_import_json_pomos, [json_pomos], [imp_json_msg, after_json_table])

    # ---- 視覺化 (進階) ----
    with gr.Tab("視覺化 (進階)"):
        vbtn = gr.Button("📊 產生圖表")
        with gr.Row():
            p1 = gr.Plot(label="圖表 1")
            p2 = gr.Plot(label="圖表 2")
            p3 = gr.Plot(label="圖表 3")
        vbtn.click(fn=ui_viz, outputs=[p1, p2, p3])

demo.launch(share=False)


Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
Note: opening Chrome Inspector may crash demo inside Colab notebooks.
* To create a public link, set `share=True` in `launch()`.


<IPython.core.display.Javascript object>